In [ ]:
import os, sys, re
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
from snowballing.operations import load_work_map_all_years, work_to_bibtex, reload
from snowballing.approaches import get_approaches, name

from IPython.display import HTML, display

reload()

all_approaches = get_approaches()
len(all_approaches)

In [ ]:
approaches = [a for a, m in all_approaches]

work_approach = {}
name_work = {}
refs = {}
order = {}
i = 0

for a in approaches:
    order[a] = i
    i += 1
    a.refs = set()
    for work in a.work:
        work_approach[work] = a

for wname, work in load_work_map_all_years():
    if work in work_approach:
        approach = work_approach[work]
        approach.refs.add(wname)
        refs[wname] = order[approach]
    name_work[wname] = work

        
def replace(matchobj):
    try:
        if matchobj.group(1) in refs:
            return '<a class="opener" href="#" data-panel="{}" title="{}">'.format(
                refs[matchobj.group(1)],
                work_to_bibtex(name_work[matchobj.group(1)], matchobj.group(1)).replace('"', "&quot;")
            )
        return '<a class="reference" href="#{}" title="{}">'.format(
            matchobj.group(1),
            work_to_bibtex(name_work[matchobj.group(1)], matchobj.group(1)).replace('"', "&quot;")
        )
    except KeyError:
        print(matchobj.group(0))
        return matchobj.group(0)

    
result = ['<div id="accordion">']
for a in approaches:
    result.append("""
        <h3> {} </h3>
        <div>
            {}
            <div>
            {}
            </div>
        </div>
    """.format(
        name(a),
        re.sub(r'<a.*?href="#(.*?)".*?>', replace, a._about),
        "\n".join(
            """<xmp style="display:inline-block;width=100px;">{}</xmp>""".format(
            work_to_bibtex(name_work[workname], workname)
        ) for workname in a.refs)
    ))
    #print([name(a), a.refs])
    
result.append("""
</div>
<script>
var $accordion = $("#accordion");

$accordion.accordion({
      collapsible: true,
      active: false,
      heightStyle: "content"
    });

$(".opener").on("click", function () {
    var $this = $(this),
        toOpen = $this.data("panel");

    $accordion.accordion("option", "active", toOpen);

    return false;
});
</script>
""")

result2 = ["""
<!doctype html>

<html lang="en">
<head>
  <meta charset="utf-8">

  <title>Provenance from Scripts Snowballing - Approaches</title>
  <link href="jquery-ui.css" rel="stylesheet">
  <meta name="description" content="Provenance from Scripts Snowballing">

  <!--[if lt IE 9]>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/html5shiv/3.7.3/html5shiv.js"></script>
  <![endif]-->
</head>

<body>
<script src="jquery-3.1.0.min.js"></script>
<script src="jquery-ui.min.js"></script>

"""] + result + ["""</body>
</html>"""]

In [ ]:
result = ['<div class="panel-group" id="accordion" role="tablist" aria-multiselectable="true">']
for i, a in enumerate(approaches):
    result.append("""
    <div class="panel panel-default">
        <div class="panel-heading" role="tab" id="heading{0}">
          <h4 class="panel-title">
            <a role="button" data-toggle="collapse" data-parent="#accordion" href="#collapse{0}" aria-expanded="false" aria-controls="collapse{0}">
            {1}
            </a>
          </h4>
        </div>
        <div id="collapse{0}" class="panel-collapse collapse" role="tabpanel" aria-labelledby="heading{0}">
          <div class="panel-body">
            {2}
            <div>
            {3}
            </div>
          </div>
        </div>
    </div>
    """.format(
        i,
        name(a),
        re.sub(r'<a.*?href="#(.*?)".*?>', replace, a._about),
        "\n".join(
            """<pre style="display:inline-block;width=100px;">{}</pre>""".format(
            work_to_bibtex(name_work[workname], workname)
        ) for workname in a.refs)
    ))
    #print([name(a), a.refs])
    
result.append("""
            </div>
        </div>
    </div> 
    <footer class="footer">
    </footer>
    <!-- jQuery (necessary for Bootstrap's JavaScript plugins) -->
    <script src="js/jquery-3.1.0.min.js"></script>
    <!-- Include all compiled plugins (below), or include individual files as needed -->
    <script src="js/bootstrap.min.js"></script>

""")

result2 = ["""

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>PinS - Provenance in Scripts</title>

    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/style.css" rel="stylesheet">

    <!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
    <!-- WARNING: Respond.js doesn't work if you view the page via file:// -->
    <!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/html5shiv/3.7.3/html5shiv.min.js"></script>
      <script src="https://oss.maxcdn.com/respond/1.4.2/respond.min.js"></script>
    <![endif]-->
  </head>
  <body>
    <nav class="navbar navbar-inverse navbar-fixed-top">
      <div class="container">
        <div class="navbar-header">
          <button type="button" class="navbar-toggle collapsed" data-toggle="collapse" data-target="#navbar" aria-expanded="false" aria-controls="navbar">
            <span class="sr-only">Toggle navigation</span>
            <span class="icon-bar"></span>
            <span class="icon-bar"></span>
            <span class="icon-bar"></span>
          </button>
          <a class="navbar-brand" href="index.html">PinS</a>
        </div>
        <div id="navbar" class="collapse navbar-collapse">
          <ul class="nav navbar-nav">
            <li><a href="index.html">Home</a></li>
            <li><a href="taxonomy.html">Taxonomy</a></li>
            <li class="active"><a href="approaches.html">Approaches</a></li>
            <li><a href="selected.html">Related Graph</a></li>
            <li><a href="graph.html">Full Graph</a></li>
          </ul>
        </div><!--/.nav-collapse -->
      </div>
    </nav>
    <div class="container">
      <div class="starter-template">

"""] + result + ["""</body>
</html>"""]

In [ ]:
display(HTML('\n'.join(result)))